In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

Procedure

- Create mesh
    - Generates object with nodes and elements as properties
    - Depends on coordinate system, mesh size, and symmetry conditions
- Add structures 
    - 

- Create mesh 
    - Depends on element size and extents
    - Returns coordinate axes (size of each is nx, ny, nz)
- Add structures to mesh
    - Get nodes in struct based on coordinates
    - Change part id of elements in struct
        - All nodes in element should be within struct? 
    -- Generate list of node ids in struct
- Add boundary constraints 
    - Depends on mesh symmetry
    - Change tc and rc of nodes on faces and edges
- Add pml
    - Depends on mesh symmetry and pml thickness
    - Change part id of non-symmetry plane elements
    - Part id + 1 if also in struct
- Finish dyna deck
    - Add material
    - Add load curves
    - Add arf load based on field params (pre-generated)
    - Add database/control parameters
        - Node sets?
- Validation functions
    - Make sure nodes exist on center axis regardless of mesh symmetry so peak arf load is modeled



- Check get_plane_node_ids to see if 0s are necessary
- Fix formatting of numbers in material cards
- Fix pml/struct interface with 4 node overlap 
    - Check len(pml_and_struct_nodes) condition is valid
- Add validation functions
    - Nodes exist on center axis regardless of mesh symmetry so peak arf load is modeled
    - Mesh extent minima are less than maxima
- Add card dictionary/dataclass to contain all dyna cards and defaults
    - Materials
    - Load curves
    - Loads (nodal forces list)
    - Database
    - Master keyword file
- Add nested and unnested folder structure write out options
    - Unnested: all files in same folder for single simulation
    - Nested: folder structure for batch simulations:
        project/mesh.dyn
               /index file 
               /load_folders/arf_load.dyn
                            /material_folders/materials.dyn
                                             /master.dyn
    - Create reader/writer for loading the index file and extracting simulation data. Use generic folder names (arf_load_x, material_x) and keep variable parameters associated with folder x in index file

In [ ]:
# @dataclass
# class Extent:
#     x: Tuple[float, float]
#     y: Tuple[float, float]
#     z: Tuple[float, float]

# @dataclass
# class Coordinates:
#     nx: int
#     ny: int
#     nz: int
#     extent: Extent
#     x: np.ndarray = field(init=False, repr=False)
#     y: np.ndarray = field(init=False, repr=False)
#     z: np.ndarray = field(init=False, repr=False)

#     def __post_init__(self):
#         self.x = np.linspace(self.extent.x[0], self.extent.x[1], self.nx)
#         self.y = np.linspace(self.extent.y[0], self.extent.y[1], self.ny)
#         self.z = np.linspace(self.extent.z[0], self.extent.z[1], self.nz)


In [67]:
from typing import List, Dict
from dataclasses import dataclass, field
from fem.dyna.material import Material

@dataclass
class Structure:
    shape: str
    material: Material
    args: list = field(default_factory=list)
    kwargs: dict = field(default_factory=dict)

struct_list = [
    Structure(
        shape = 'rectangle', 
        args = [-1, -0.5, -0.5, -0, -4, -2],
        material = mat2,  
    ), 
    Structure(
        shape = 'sphere', 
        # args = [0, 0, -2, 1],
        kwargs = {'xc': 0, 'yc': 0, 'zc': -2, 'radius': 1},
        material = mat2,  
    ), 
]

for s in struct_list:
    print(s.kwargs)

{}
{'xc': 0, 'yc': 0, 'zc': -2, 'radius': 1}


In [2]:
from fem.dyna.mesh import Coordinates, DynaMesh
from fem.dyna._structure import Structure
from fem.dyna.material import KelvinMaxwellViscoelastic

mat = KelvinMaxwellViscoelastic(density=1, E=26.11, nu=0.499, eta=2.34)
mat2 = KelvinMaxwellViscoelastic(density=1, E=2*26.11, nu=0.499, eta=2.34)

# struct_list = [
#     {
#         'shape': 'rectangle', 
#         'struct_args': [-1, -0.5, -0.5, -0, -4, -2],
#         'material': mat2,  
#     }, 
#     {
#         'shape': 'sphere', 
#         'struct_args': [0, 0, -2, 1],
#         'struct_kwargs': {'xc': 0, 'yc': 0, 'zc': -2, 'radius': 1},
#         'material': mat2,  
#     }, 
# ]

struct_list = [
    Structure(
        shape = 'rectangle', 
        args = [-1, -0.5, -0.5, -0, -4, -2],
        material = mat2,  
    ), 
    Structure(
        shape = 'sphere', 
        args = [0, 0, -2, 1],
        # kwargs = {'xc': 0, 'yc': 0, 'zc': -2, 'radius': 1},
        material = mat2,  
    ), 
]

coords = Coordinates(
    nx=11, ny=11, nz=11,
    xmin=-1.0, xmax=0.0,
    ymin=-1.0, ymax=0.0,
    zmin=-4.0, zmax=0.0
)
coords.x
symmetry = 'q'
mesh = DynaMesh(coords, symmetry, mat)

mesh.constrain_boundary_nodes()
mesh.add_pml(pml_thickness=2)
mesh.add_struct_list(struct_list)

mesh.set_control(end_time=4.5e-3)
mesh.set_database(dt=2e-5)
mesh.set_master(title='testing')

project_path = r"C:\Users\joeyr\repos\fem\sims"
load_folder_name = 'load_0'
material_folder_name = 'material_0'
mesh.write_all_dyna_cards(project_path, load_folder_name, material_folder_name)


NameError: name 'Structure' is not defined

In [49]:
def format_dyna_number(num):
    snum = str(num)

    if len(snum) > 10:
        # Convert number to scientific notation
        snum = "{:E}".format(num)

        # If number string is greater than 10 characters, remove leading zeros from exponent
        if len(snum) > 10:
            base, exponent = snum.split('E')
            sign = exponent[0]
            exponent = exponent[1:].lstrip('0')
            snum = base + 'E' + sign + exponent

        # If number string is still greater than 10 characters, remove precision bits to make it 10 characters long
        if len(snum) > 10: 
            base, exponent = snum.split('E')
            precision_bits_to_remove = len(snum) - (10 - len(exponent) + 2)
            snum = base[:-precision_bits_to_remove] + 'E' + exponent

    return snum


print(format_dyna_number(-98765432.1))
print(format_dyna_number(-0.00000000123456789))
print(format_dyna_number(12345.6789))

-9.8765E+7
-1.2345E-9
12345.6789


In [ ]:
np.unique(mesh.elems['pid'])

In [ ]:
def format_dyna_scientific(s):
    s = f"{s:>20.4E}"
    if 'E+' in s:
        s = f" {s.replace('E+0', 'E+')}" 
    elif 'E-' in s:
        s = f" {s.replace('E-0', 'E-')}"
    return s

t_arf = 70e-6
dt = 5e-6
tracks_between = 3
prf = 10e3
t_delay = (tracks_between + 1)/prf
n_arf = 3

t = []
for iarf in range(n_arf):
    narf_delay = iarf*t_delay
    t.extend([
        [narf_delay, 0],
        [narf_delay+dt, 1],
        [narf_delay+dt+t_arf, 1],
        [narf_delay+2*dt+t_arf, 0],
    ])

for a, o in t:
    s = format_dyna_scientific(a) + format_dyna_scientific(o)
    print(s)
